In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import streamlit as st

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint


import kaggle

In [4]:
kaggle.api.authenticate()
dataset_url = 'ravirajsinh45/real-life-industrial-dataset-of-casting-product'
kaggle.api.dataset_download_files(dataset_url, unzip=True)


Dataset URL: https://www.kaggle.com/datasets/ravirajsinh45/real-life-industrial-dataset-of-casting-product


In [5]:
train_generator = ImageDataGenerator(rotation_range = 360,
                                     width_shift_range = 0.05,
                                     height_shift_range = 0.05,
                                     shear_range = 0.05,
                                     zoom_range = 0.05,
                                     horizontal_flip = True,
                                     vertical_flip = True,
                                     brightness_range = [0.75, 1.25],
                                     rescale = 1./255,
                                     validation_split = 0.2)

In [6]:
# get data 
#menentukan satu set nilai lain untuk parameter 'flow_from_directory':
IMAGE_DIR = "./casting_data/casting_data/"
IMAGE_SIZE = (300, 300)
BATCH_SIZE = 64
SEED_NUMBER = 123

gen_args = dict(target_size = IMAGE_SIZE,
                color_mode = "grayscale",
                batch_size = BATCH_SIZE,
                class_mode = "binary",
                classes = {"ok_front": 0, "def_front": 1},
                seed = SEED_NUMBER)

train_dataset = train_generator.flow_from_directory(
                                        directory = IMAGE_DIR + "train",
                                        subset = "training", shuffle = True, **gen_args)
validation_dataset = train_generator.flow_from_directory(
                                        directory = IMAGE_DIR + "train",
                                        subset = "validation", shuffle = True, **gen_args)

Found 5307 images belonging to 2 classes.
Found 1326 images belonging to 2 classes.


In [7]:
mapping_class = {0: "normal", 1: "defect"}
mapping_class

{0: 'normal', 1: 'defect'}

In [8]:
def visualizeImageBatch(dataset, title):
    images, labels = next(iter(dataset))
    images = images.reshape(BATCH_SIZE, *IMAGE_SIZE)
    fig, axes = plt.subplots(8, 8, figsize=(16,16))

    for ax, img, label in zip(axes.flat, images, labels):
        ax.imshow(img, cmap = "gray")
        ax.axis("off")
        ax.set_title(mapping_class[label], size = 20)

    plt.tight_layout()
    fig.suptitle(title, size = 30, y = 1.05, fontweight = "bold")
    plt.show()
    
    return images

In [10]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=3, strides=2, activation="relu", input_shape=IMAGE_SIZE + (1, )))
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Conv2D(filters=16, kernel_size=3, strides=2, activation="relu"))
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(rate=0.2))
model.add(Dense(64, activation="relu"))
model.add(Dropout(rate=0.2))
model.add(Dense(1, activation="sigmoid"))

In [11]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 149, 149, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 36, 36, 16)     │         4,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 18, 18, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 5184)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │       663,680 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 676,945 (2.58 MB)

 Trainable params: 676,945 (2.58 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [13]:
checkpoint = ModelCheckpoint('model/cnn_casting_inspection_model.keras', verbose=1, save_best_only=True, monitor='val_loss', mode='min')

model.fit(train_dataset, validation_data=validation_dataset, batch_size=16, epochs=10, callbacks=[checkpoint], verbose=1)


Epoch 1/10


C:\Users\opiks\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5506 - loss: 0.6853
Epoch 1: val_loss improved from inf to 0.61211, saving model to model/cnn_casting_inspection_model.keras
83/83 ━━━━━━━━━━━━━━━━━━━━ 132s 1s/step - accuracy: 0.5509 - loss: 0.6852 - val_accuracy: 0.6772 - val_loss: 0.6121
Epoch 2/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 892ms/step - accuracy: 0.6747 - loss: 0.5840
Epoch 2: val_loss improved from 0.61211 to 0.50337, saving model to model/cnn_casting_inspection_model.keras
83/83 ━━━━━━━━━━━━━━━━━━━━ 91s 1s/step - accuracy: 0.6748 - loss: 0.5838 - val_accuracy: 0.7270 - val_loss: 0.5034
Epoch 3/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 842ms/step - accuracy: 0.7264 - loss: 0.5135
Epoch 3: val_loss improved from 0.50337 to 0.45962, saving model to model/cnn_casting_inspection_model.keras
83/83 ━━━━━━━━━━━━━━━━━━━━ 86s 987ms/step - accuracy: 0.7265 - loss: 0.5134 - val_accuracy: 0.7572 - val_loss: 0.4596
Epoch 4/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 766ms/step - accuracy: 0.7766 - loss: 0.448

In [14]:
model.save("model.h5")

In [28]:
tf.keras.models.save_model(model,'/home/taufiqu/Capstone37')

2024-05-27 21:47:09.306242: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,128]
	 [[{{node inputs}}]]
2024-05-27 21:47:09.380997: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,64]
	 [[{{node inputs}}]]
2024-05-27 21:47:11.038871: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,128]
	 [[{{node inputs}}]]
2024-05-27 21:47:

INFO:tensorflow:Assets written to: /home/taufiqu/Capstone37/assets


INFO:tensorflow:Assets written to: /home/taufiqu/Capstone37/assets
